In [1]:
import csv
import glob
import os
import shutil

In [2]:
RAW_DATA_PATH = '../mount'

NEG_DIRS = ['Non referral1', 'Non Referral2', 'NonReferral3']
POS_DIRS = ['Referral1', 'Referral2', 'Referral3']

anonymize_mapping_file = 'mapping.csv'

In [3]:
# verify case (anonymized) and patient IDs are a one-to-one mapping
# verify patient ID in each visit matches patient ID in parent folder name

case_to_patient = {}
patient_to_case = {}

for label, top_dirs in [('pos', POS_DIRS), ('neg', NEG_DIRS)]:
    for top_dir in top_dirs:
        top_dir_path = os.path.join(RAW_DATA_PATH, top_dir)
        print(label, top_dir_path)
        for case_dir in os.listdir(top_dir_path):
            case_dir_path = os.path.join(top_dir_path, case_dir)
            print(case_dir)
            
            for visit_dir in os.listdir(case_dir_path):
                visit_dir_path = os.path.join(case_dir_path, visit_dir)
                patient_id, date = visit_dir.split()
                print(patient_id)
                if case_dir in case_to_patient:
                    assert case_to_patient[case_dir] == patient_id
                else:
                    case_to_patient[case_dir] = patient_id
                
                if patient_id in patient_to_case:
                    assert patient_to_case[patient_id] == case_dir
                else:
                    patient_to_case[patient_id] = case_dir
                    
                for file in os.listdir(visit_dir_path):
                    patient_id_2, *rest = file.split('_')
                    assert patient_id == patient_id_2

pos ../mount/Referral1
Case100
P22671788
P22671788
P22671788
P22671788
P22671788
P22671788
P22671788
P22671788
Case102
P26500017
P26500017
Case109
P30230643
P30230643
P30230643
P30230643
Case113
P02045359
P02045359
P02045359
P02045359
Case115
P25461534
Case116
P26315069
Case123
P23081870
P23081870
Case125
P08085722
P08085722
Case133
P22636658
P22636658
P22636658
P22636658
P22636658
P22636658
Case135
P23586498
Case139
P33649153
Case152
P30698260
Case156
P23593833
Case158
P21746722
Case170
P17554148
Case172
P11921285
P11921285
P11921285
P11921285
Case28
P08159394
P08159394
Case36
P08908014
Case39
P23723182
P23723182
Case43
P21497508
P21497508
P21497508
P21497508
Case45
P11775111
P11775111
P11775111
P11775111
P11775111
Case46
P35954221
Case48
P36238533
Case50
P10885853
P10885853
Case51
P09076431
P09076431
P09076431
P09076431
P09076431
P09076431
P09076431
P09076431
P09076431
Case64
P30882567
Case69
P21501374
Case70
P29245909
Case73
P36963247
Case74
P21361258
P21361258
P21361258
P21361258
P

In [4]:
with open(os.path.join(RAW_DATA_PATH, anonymize_mapping_file), 'w') as f:
    csvwriter = csv.writer(f)
    for case, patient in case_to_patient.items():
        assert patient_to_case[patient] == case
        csvwriter.writerow([case, patient])

In [ ]:
# run to anonymize
for label, top_dirs in [('pos', POS_DIRS), ('neg', NEG_DIRS)]:
    for top_dir in top_dirs:
        top_dir_path = os.path.join(RAW_DATA_PATH, top_dir)
        print('=================')
        print(top_dir_path)
        for case_dir in os.listdir(top_dir_path):
            case_dir_path = os.path.join(top_dir_path, case_dir)
            print(case_dir)
            
            for visit_dir in os.listdir(case_dir_path):
                visit_dir_path = os.path.join(case_dir_path, visit_dir)
                patient_id, date = visit_dir.split()
                print(patient_id)
                    
                for file in os.listdir(visit_dir_path):
                    patient_id_2, *rest = file.split('_')
                    assert patient_id == patient_id_2
                    new_file = '_'.join([case_dir] + rest)
                    
                    old_path = os.path.join(visit_dir_path, file)
                    new_path = os.path.join(visit_dir_path, new_file)
                    print(f'Renaming: {old_path} -> {new_path}')
                    try:
                        os.rename(old_path, new_path)
                    except e:
                        print('!!!!!! ERROR WHEN RENAMING FILE !!!!!!')
                        print(e)
                    break
    
                # Google Cloud Storage does not support 'mv'
                new_dir_path = os.path.join(case_dir_path, f'{case_dir} {date}')
                print(f'Copying dir: {visit_dir_path} -> {new_dir_path}')
                try:
                    shutil.copytree(visit_dir_path, new_dir_path)
                except e:
                    print('!!!!!! ERROR WHEN COPYING DIR !!!!!!')
                    print(e)
                    
                print(f'Deleting dir: {visit_dir_path}')
                try:
                    shutil.rmtree(visit_dir_path)
                except e:
                    print('!!!!!! ERROR WHEN DELETING DIR !!!!!!')
                    print(e)
            break
        break
    break